In [1]:
import geopandas as gpd
from geojson_pydantic import LineString
from matplotlib.cbook import pts_to_midstep
from networkx import neighbors
from numpy.ma.core import append

gpd.read_file('blocks (1).geojson')

ModuleNotFoundError: No module named 'networkx'

In [ ]:
# Пример вызова генплана на большой территории 

import geopandas as gpd
from shapely import Polygon, MultiPolygon

from app.gen_planner.python.src import GenPlanner, gen_plan, basic_func_zone

zones, roads = GenPlanner(gpd.read_file('1.gpkg')).poly2func2terr2block(gen_plan)

In [ ]:
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', categorical=True)
m1

In [ ]:
# тут вызовы по сценариям по разным территориям, везде basic по умолчанию, но другие лежат в zoning.


from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
# тут вызов деления территориальной зоны на кварталы по минимальному размеру терр.зоны
from app.gen_planner.python.src import GenPlanner, terr_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).features2blocks(terr_zones.residential_terr)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner
import geopandas as gpd

zones = {'1': 0.1,
         '2': 0.1,
         '3': 0.1,
         '4': 0.1,
         '5': 0.1,
         '6': 0.1,
         '7': 0.1,
         '8': 0.1, }
zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).split_features(zones, roads_width=50)
zones.explore()

In [ ]:
import geopandas as gpd
import numpy as np
from shapely import MultiPolygon, Polygon

local_crs = 32636
multip = gpd.read_file('2.gpkg').to_crs(local_crs)


def elasatic_wrap(gdf: gpd.GeoDataFrame) -> Polygon:
    gdf = gdf.copy()
    multip = gpd.GeoDataFrame(geometry=gdf.geometry, crs=gdf.crs).explode(ignore_index=True)
    max_dist = np.ceil(multip.apply(lambda row: multip.drop(row.name).distance(row.geometry).min(), axis=1).max(axis=0))
    poly = multip.buffer(max_dist).union_all().buffer(-max_dist)
    if isinstance(poly, MultiPolygon):
        raise RuntimeError('Cant cast Multipolygon to Polygon')
    poly = Polygon(poly.exterior)
    return poly


p = elasatic_wrap(multip)

In [ ]:
gpd.GeoDataFrame(geometry=[p.simplify(250)], crs=local_crs).explore()

In [ ]:
from app.gen_planner.python.src import GenPlanner, gen_plan
import geopandas as gpd

gp = GenPlanner(gpd.read_file('2.gpkg'), rotation=0)
multiterritory = gp.territory_to_work_with
zones, roads = gp.poly2func2terr2block(gen_plan)

In [ ]:
zones.explore()

# MultiP

In [ ]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
from app.gen_planner.python.src.utils.geom_utils import geometry_to_multilinestring
from shapely.ops import polygonize, unary_union, Polygon


def territory_splitter(gdf_to_split: gpd.GeoDataFrame, splitters: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    local_crs = gdf_to_split.estimate_utm_crs()
    gdf_to_split = gdf_to_split.to_crs(local_crs)
    splitters = splitters.to_crs(local_crs)
    lines_orig = gdf_to_split.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()
    lines_splitters = splitters.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()

    polygons = gpd.GeoDataFrame(geometry=list(polygonize(unary_union(lines_orig + lines_splitters))),
                                crs=local_crs).clip(gdf_to_split.to_crs(local_crs), keep_geom_type=True).explode()

    polygons_points = polygons.copy()
    polygons_points.geometry = polygons.representative_point()
    joined_ind = polygons_points.sjoin(splitters, how='inner', predicate='within').index.tolist()
    polygons['is_splitter'] = polygons.index.isin(joined_ind)
    polygons.geometry = polygons.apply(lambda x: Polygon(x.geometry.exterior) if not x['is_splitter'] else x.geometry,
                                       axis=1)
    non_splitters = polygons[~polygons['is_splitter']]
    contains = non_splitters.sjoin(non_splitters, predicate='contains')
    to_kick = contains[~(contains.index == contains['index_right'])]['index_right'].to_list()
    polygons.drop(index=to_kick, inplace=True)
    return polygons


local_crs = 32636

territory = gpd.read_file('project ШлиссельбургMulti.geojson').to_crs(local_crs)
roads = gpd.read_file('roads_shlis.geojson').to_crs(local_crs)
roads.geometry = roads.to_crs(32636).buffer(2.5).to_crs(4326)

multipolys = territory_splitter(territory, roads)
multipolys.explore(style_kwds={'fillOpacity': 0.5})

In [ ]:
from app.gen_planner.python.src import func_zones, GenPlanner

multipolys = multipolys[~multipolys['is_splitter']]
zones, roads = GenPlanner(multipolys,dev_mod=True).features2terr_zones(func_zones.residential_func_zone)

In [ ]:
multipolys.reset_index(drop=True, inplace=True)

In [ ]:
local_crs = 32636

lines_orig = multipolys.geometry.apply(geometry_to_multilinestring).to_list()
lines_new = zones.geometry.apply(geometry_to_multilinestring).to_list()
polygons = gpd.GeoDataFrame(geometry=list(polygonize(unary_union(lines_orig + lines_new))),
                            crs=local_crs).explode()
polygons['subsplit_area'] = polygons.area
polygons_points = polygons.copy()
polygons_points.geometry = polygons_points.representative_point()
polygons_points = polygons_points.sjoin(zones, how='inner', predicate='within').drop(columns='index_right')
multipolys['original_area'] = multipolys.area
division = multipolys.sjoin(polygons_points, how='inner', predicate='intersects').drop(
    columns=['index_right', 'is_splitter', 'ratio'])

division['zone_to_add'] = division['zone_name'].apply(lambda x: x.name)
division = division.reset_index().groupby(['index', 'zone_to_add']).agg(
    {'original_area': 'first', 'subsplit_area': 'sum', 'zone_name': 'first',
     'geometry': 'first'}).reset_index().set_index(
    'index')
division

In [ ]:
import pandas as pd

target_zones = pd.DataFrame.from_dict(func_zones.residential_func_zone.zones_ratio, orient='index',
                                      columns=['zones_ratio']).reset_index().rename(columns={'index': 'zone'})
target_zones['zones_ratio'] = target_zones['zones_ratio'] / target_zones['zones_ratio'].sum()
target_zones['min_area'] = target_zones['zone'].apply(lambda x: x.min_block_area)
target_zones['zone'] = target_zones['zone'].apply(lambda x: x.name)
target_zones.set_index('zone', inplace=True)

possible_areas = division.reset_index().groupby('index').agg(
    {'zone_to_add': list, 'subsplit_area': list})
possible_areas = possible_areas.join(
    possible_areas.apply(lambda x: pd.Series([y for y in x.subsplit_area], index=x.zone_to_add), axis=1))
possible_areas = possible_areas.drop(columns=['zone_to_add', 'subsplit_area'])
possible_areas['poly_area'] = possible_areas[target_zones.index].sum(axis=1)
target_zones['target_area'] = target_zones['zones_ratio'] * possible_areas['poly_area'].sum()
target_zones

In [ ]:
import pandas as pd
import pulp

# === 1. Входные данные ===

# Допустимые размещения
df_zones = division.copy()  # или подставь свой путь/данные напрямую
df_zones = df_zones.reset_index()
# Целевые показатели
df_targets = target_zones.copy()
df_targets = df_targets.reset_index()
# Приводим данные в подходящий вид
df_zones['original_area'] = df_zones['original_area'].astype(float)

# Собираем уникальные комбинации (index, zone_to_add)
valid_combinations = df_zones[['index', 'zone_to_add', 'original_area']].drop_duplicates()
# Получим словари для удобства
zone_capacity = df_zones.groupby('index')['original_area'].first().to_dict()
zone_permitted = {(row['index'], row['zone_to_add']) for _, row in valid_combinations.iterrows()}
min_areas = df_targets.set_index('zone')['min_area'].to_dict()
target_areas = df_targets.set_index('zone')['target_area'].to_dict()

model = pulp.LpProblem("Territorial_Zoning", pulp.LpMinimize)

x = {
    (i, z): pulp.LpVariable(f"zone_index {i} zone type {z}", lowBound=0)
    for (i, z) in zone_permitted
}

y = {
    (i, z): pulp.LpVariable(f"y_{i}_{z}", cat="Binary")
    for (i, z) in zone_permitted
}
slack = {
    (i, z): pulp.LpVariable(f"slack_{i}_{z}", lowBound=0)
    for (i, z) in x
}

model += 0

In [ ]:

# (1) Ограничение по площади каждой зоны
for i in df_zones['index'].unique():
    model += pulp.lpSum(x[i, z] for z in df_targets['zone'] if (i, z) in x) <= zone_capacity[i], f"Capacity_zone_{i}"

for (i, z) in x:
    model += x[i, z] + slack[i, z] >= min_areas[z] * y[i, z], f"SoftMinArea_{i}_{z}"
    model += x[i, z] <= zone_capacity[i] * y[i, z], f"MaxIfAssigned_{i}_{z}"

# (3) Обеспечение целевой площади для каждой тер. зоны
for z in df_targets['zone']:
    model += pulp.lpSum(x[i, z] for i in df_zones['index'].unique() if (i, z) in x) >= target_areas[
        z], f"TargetArea_{z}"


In [ ]:
model

In [ ]:

solver = pulp.PULP_CBC_CMD(msg=True)
model.solve(solver)

# === 5. Вывод результатов ===

print("Статус:", pulp.LpStatus[model.status])

allocations = []
for (i, z), var in x.items():
    val = var.varValue
    if val and val > 0:
        allocations.append((i, z, round(val, 2)))

df_result = pd.DataFrame(allocations, columns=["zone_index", "territorial_zone", "assigned_area"])
df_result


In [ ]:
df_result['zone_index'].unique()

In [ ]:
from app.gen_planner.python.src import func_zones, GenPlanner
from app.gen_planner.python.src import FuncZone, terr_zones

result = []
for ind, poly in multipolys.loc[df_result['zone_index'].unique()].iterrows():
    terr_zones_in_poly = df_result[df_result['zone_index'] == ind].copy()
    terr_zones_in_poly = terr_zones_in_poly[terr_zones_in_poly['assigned_area'] > 0]
    if len(terr_zones_in_poly) == 1:
        result.append((poly.geometry, terr_zones_in_poly.iloc[0]['territorial_zone']))
    else:
        zone_area_total = poly.original_area
        zones_ratio_dict = {
            row['territorial_zone']: row['assigned_area'] / zone_area_total
            for _, row in terr_zones_in_poly.iterrows()
        }
        subzones, roads = GenPlanner(multipolys.loc[[ind]]).split_features(zones_ratio_dict=zones_ratio_dict)
        for _, zone in subzones.iterrows():
            result.append((zone.geometry, zone.zone_name))

In [ ]:
gpd.GeoDataFrame(data=result, columns=['geometry', 'zone_name'], geometry='geometry', crs=32636).explore(
    column='zone_name',  cmap='Dark2',tiles='cartodb positron')

In [ ]:
zones['zone'] = zones['zone_name'].apply(lambda x: x.name)

In [ ]:
m1 = multipolys.reset_index().explore(style_kwds={'fillOpacity': 0.2}, tiles='cartodb positron')
zones.explore(m=m1, column='zone', tiles='CartoDB positron', cmap='Dark2', categorical=True,
              style_kwds={'fillOpacity': 0.4})
# zones2.explore(m=m1)
m1

In [ ]:
multipolys.reset_index().explore(style_kwds={'fillOpacity': 0.2}, tiles='cartodb positron')